In [1]:
from music21 import *
import math
import music21
import numpy as np
import pandas as pd

In [2]:
score = converter.parse('./Chopin__Trois_Valses.mxl')
# score.show('musicxml')

In [3]:
beats = score.getTimeSignatures(recurse=True)[0].numerator

measures = int(math.ceil(score.highestTime / beats)) + 1
print(measures)


505


In [4]:
durations = []
is_chord = []
for i in range(1, measures, 1):
    measure = score.parts[0].measure(i).flatten().notesAndRests
    lengths = []
    for j in range(0, len(measure)):
        s = measure[j].duration.quarterLength
        is_chord.append(False)
        lengths.append(float(s))
    durations.append(lengths)

In [5]:
measure_notes = []
measure_notes_frequency = []
for i in range(1, measures, 1):
    measure = score.parts[0].measure(i).flatten().notesAndRests
    notes = []
    notes_frequency = []
    for j in range(0, len(measure)):
        if (measure[j].isChord):
            chord = measure[j].notes
            notes.append(str(chord[-1].pitch.name + str(chord[-1].pitch.octave)))
            notes_frequency.append(chord[-1].pitch.frequency)
            continue
        elif (measure[j].isRest):
            s = 'rest'
            f = 0.0
        else:
            f = measure[j].pitch.frequency
            s = str(measure[j].pitch.name)
            s += str(measure[j].pitch.octave)
        notes.append(s)
        notes_frequency.append(f)
    measure_notes.append(notes)
    measure_notes_frequency.append(notes_frequency)

In [6]:
bpm = 260
quarter_note_duration = (1 / bpm) * 60
note_duration = []
for measure in durations:
    note_duration.append([note_length * quarter_note_duration for note_length in measure])
# note_duration

In [7]:
new_durations = np.concatenate(durations)
new_measure_notes = np.concatenate(measure_notes)
new_measure_notes_frequency = np.concatenate(measure_notes_frequency)
new_note_duration = np.concatenate(note_duration)

start_times = []
start_times.append(0)
curr_time = 0

for i in range(0, len(new_note_duration) - 1):
    curr_time += new_note_duration[i]
    start_times.append(curr_time)

note_type = [duration.Duration(quarterLength=quarter_note_length).type for quarter_note_length in new_durations]

In [8]:
df = pd.DataFrame({'Note Type': note_type, 'Note Duration (Seconds)': new_note_duration, 
                   'Note Name': new_measure_notes, 'Note Frequency': new_measure_notes_frequency, 
                   'Start Time': start_times})

In [9]:
df

,Note Type,Note Duration (Seconds),Note Name,Note Frequency,Start Time
0,quarter,0.230769,A-4,415.304698,0.000000
1,eighth,0.115385,G4,391.995436,0.230769
2,eighth,0.115385,A-4,415.304698,0.346154
3,eighth,0.115385,C5,523.251131,0.461538
4,eighth,0.115385,B-4,466.163762,0.576923
...,...,...,...,...,...
2592,eighth,0.115385,A-5,830.609395,391.500000
2593,eighth,0.115385,E-5,622.253967,391.615385
2594,eighth,0.115385,C5,523.251131,391.730769
2595,eighth,0.115385,A-4,415.304698,391.846154
